<a href="https://colab.research.google.com/github/mahopman/IEBM-Net/blob/main/evidence_integration/generate_evidence_integration_drug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
__author__ = 'Qiao Jin'
__editor__ = 'Mia Hopman'

In [19]:
local_path = '/IEBM-Net_Data'
evidence_integration_path = f'{local_path}/evidence_integration'

In [8]:
import json

def generate(picoids, split_path):
	output = []
	for picoid in picoids:
		if prompt_info[picoid]['label'] != 'invalid prompt':
			output.append({})
			output[-1]['picoid'] = picoid
			output[-1]['pmcid'] = prompt_info[picoid]['PMCID']
			output[-1]['i_text'] = prompt_info[picoid]['I']
			output[-1]['c_text'] = prompt_info[picoid]['C']
			output[-1]['o_text'] = prompt_info[picoid]['O']
			output[-1]['label'] = result2label[prompt_info[picoid]['label']]

			passage = ''
			if str(prompt_info[picoid]['PMCID']) in pmcid2content:
				content = pmcid2content[str(prompt_info[picoid]['PMCID'])]
				for secname, text in content:
					if secname[:len('ABSTRACT')] != 'ABSTRACT': continue
					if sec2label[secname2sec[secname]] == '1':
						passage += text

			output[-1]['passage'] = passage

	with open(split_path, 'w') as f:
		json.dump(output, f, indent=4)

In [12]:
result2label = {'significantly decreased': 0,\
				'no significant difference': 1,\
				'significantly increased': 2}

prompt_info = json.load(open(f'{evidence_integration_path}/evidence_inference/prompt_info.json'))
split2ids = json.load(open(f'{evidence_integration_path}/evidence_inference/split2ids.json'))
pmcid2picoid = json.load(open(f'{evidence_integration_path}/evidence_inference/pmcid2picoid.json'))
pmcid2content = json.load(open(f'{evidence_integration_path}/evidence_inference/pmc_contents.json'))
secname2sec = json.load(open(f'{evidence_integration_path}/evidence_inference/secname2sec.json'))
sec2label = json.load(open(f'{evidence_integration_path}/evidence_inference/sec2label.json'))
intervention2id = json.load(open(f'{evidence_integration_path}/intervention2ids.json'))

drug_pmcids = intervention2id['DRUG']

for split, ids in split2ids.items():
    picoids = []

    for pmcid in ids:
        pmcid = str(pmcid)
        if pmcid in pmcid2picoid and pmcid in drug_pmcids:
            picoids += pmcid2picoid[pmcid]

    split_path = f"{evidence_integration_path}/{split}.json"
    generate(picoids, split_path)